In [ ]:
import sys
import os

import numpy as np
import matplotlib.pyplot as plt
import glob
import pandas as pd
import seaborn as sns
import pickle

import scanpy as sc
import scanpy.external as sce
import anndata as ad

import session_info
import warnings
from pyprojroot.here import here

import scienceplots
from sklearn.neighbors import NearestNeighbors
from typing import Iterable

warnings.filterwarnings("ignore")
sys.path.insert(1, str(here('bin')))

# Import custom functions
from customPythonFunctions import cumulative_explained_variance, balanced_sample, generateID2SymbolDF

print("Main directory path: {}".format(here()))

plt.style.use(['nature'])
dpi_fig_save = 300
sc.set_figure_params(dpi=100, dpi_save=dpi_fig_save, vector_friendly=True)

**Setting parameters** 

In [ ]:
overwriteFigures = True
overwriteData = True

In [ ]:
cellGroup = 'B'
annotationDFpath = '/scratch_isilon/groups/singlecell/shared/projects/Inflammation-PBMCs-Atlas/02_cell_annotation/01_fromDatasets_to_CellLineages/results/INFLAMMATION_cluster_res0.1_annotated.pkl'
annotationColumns = 'cell_lineages'
workDir = os.getcwd()
level = workDir.split(os.path.sep)[-2]

**Loading the integrated object**

In [ ]:
adataHVGint = sc.read_h5ad(here('{}/results/02_{}_{}_HVGsubset_scVI.h5ad'.format(workDir, level, cellGroup)))
adataHVGint

# Computing the projections of the embedded space

## Computing KNN graph and UMAP from the integrated embedded space

In [ ]:
n_latent = adataHVGint.obsm['X_scVI'].shape[1]
sc.pp.neighbors(adataHVGint, n_neighbors=10, n_pcs=n_latent, use_rep = 'X_scVI', key_added = 'scVI')

In [ ]:
sc.tl.umap(adataHVGint, neighbors_key='scVI')

In [ ]:
# Saving the umap computed on the scVI latent component
adataHVGint.obsm['X_umap_scVI'] = adataHVGint.obsm['X_umap'].copy()
adataHVGint.uns['umap_scVI'] = adataHVGint.uns['umap']

In [ ]:
adataHVGint

**Plotting parameters** 

In [ ]:
max_cells_plot = 200000
fraction = min(1, (max_cells_plot/adataHVGint.shape[0]))

### Feature scaling and centering

In [ ]:
adataHVGint.X = adataHVGint.layers['log1p_10e4_counts']
sc.pp.scale(adataHVGint, zero_center = True)

**Loading the full object before HVG subsetting**

This will allows us to inspect genes that are not included in the HV set

In [ ]:
adata_stepAll = ad.read_h5ad(here('{}/results/01_{}_normalized.h5ad'.format(workDir, cellGroup)))
adata_stepAll

In [ ]:
# Load the corresponding annotation
annotationDF = pd.read_pickle(here(annotationDFpath))
idxSelCells = annotationDF.index[annotationDF[annotationColumns] == cellGroup].tolist()
adata = adata_stepAll[idxSelCells,:]
adata

In [ ]:
del adata_stepAll

**Merging the objects**

We will transfer the following field to the fullObject: uns, obsm, and obsp. We will also transfer the HVG set

In [ ]:
adata.uns = adataHVGint.uns.copy()
adata.obsm = adataHVGint.obsm.copy()
adata.obsp = adataHVGint.obsp.copy()
adata.var['highly_variable'] = False
adata.var['highly_variable'][adata.var_names.isin(adataHVGint.var_names)] = True

In [ ]:
# Now we can remove from memory the subsetted adata object
del adataHVGint

# Save object

In [ ]:
if overwriteData:
    adata.write(here("{}/results/03_{}_scVI_UMAP_clinical_allGenes.h5ad".format(workDir, cellGroup)), compression="gzip")

# Evaluate integration

At this point, it is important to evaluate the integration step to ensure the batch-effect have been removed, and the biological variance has been preserved. Here, we visually inspect some quality control metrics.

In [ ]:
### To reduce the computational time, we are plotting only 10% of cells
SScells = balanced_sample(adata.obs, cols = ['sampleID'], n = None, frac = fraction, shuffle = True, random_state = 42)['cellID']
adataSS = adata[SScells].copy()
del adata
adataSS

## Technical confounding factors

In [ ]:
fig = sc.pl.embedding(adataSS, basis = 'X_umap_scVI', vmin='p1', vmax='p99', 
                 color=['studyID', 'chemistry', 'technology',
                        'total_counts', 'n_genes_by_counts', 
                        'pct_counts_mt', 'pct_counts_ribo', 'pct_counts_hb', 'pct_counts_plt', 
                       'doublet_score'], 
                 wspace = 0.4, ncols = 4, return_fig=True)

if overwriteFigures:
    plt.savefig(here('{}/results/figures/03_UMAP_technicalFactors_scVI.pdf'.format(workDir)), 
                bbox_inches='tight', pad_inches=0, dpi=dpi_fig_save)

## Clinical covariates

In [ ]:
fig = sc.pl.embedding(adataSS, basis = 'X_umap_scVI', vmin='p1', vmax='p99',
                 color=['sex','binned_age', 'disease',
                       "smokingStatus", "ethnicity", "therapyResponse"], 
                 wspace = 0.4, ncols = 3, return_fig=True)

if overwriteFigures:
    plt.savefig(here('{}/results/figures/03_UMAP_clinicalFactors_scVI.pdf'.format(workDir)), 
                bbox_inches='tight', pad_inches=0, dpi=dpi_fig_save)

## Cell cycling score

In [ ]:
fig = sc.pl.embedding(adataSS, basis = 'X_umap', vmin='p1', vmax='p99',
                 ncols=3, color=['S_score', 'G2M_score', 'phase'], wspace = 0.4, return_fig=True)

if overwriteFigures:
    plt.savefig(here('{}/results/figures/03_UMAP_cellcyclingScore_scVI.pdf'.format(workDir)), 
                bbox_inches='tight', pad_inches=0, dpi=dpi_fig_save)

In [ ]:
session_info.show()